In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df=df.dropna()

In [6]:
X=df.drop('label',axis=1)

In [7]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [8]:
y=df['label']

In [9]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense

In [14]:
vocab_size=5000

**One Hot Representation**

In [15]:
messages=X.copy()

In [16]:
messages.reset_index(inplace=True)

In [17]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vaibhav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [21]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [22]:
onehot=[one_hot(words,vocab_size) for words in corpus]

In [23]:
onehot[0]

[3162, 1914, 3925, 1633, 1015, 1126, 1128, 808, 4472, 2677]

**Embedded Representation**

In [24]:
sentence_length=20
embedded=pad_sequences(onehot,padding='pre',maxlen=sentence_length)

In [25]:
embedded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3162,
       1914, 3925, 1633, 1015, 1126, 1128,  808, 4472, 2677])

In [27]:
embedding_vector=40

**Creating Model**

In [27]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector,input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


**Adding Dropout**

In [29]:
from tensorflow.keras.layers import Dropout
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector,input_length=sentence_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [31]:
len(embedded)

18285

In [32]:
y.shape

(18285,)

In [33]:
import numpy as np

In [34]:
X_final=np.array(embedded)
y_final=np.array(y)

In [35]:
X_final.shape

(18285, 20)

In [36]:
y_final.shape

(18285,)

In [37]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [38]:
X_train.shape

(12250, 20)

In [39]:
y_train.shape

(12250,)

In [40]:
X_test.shape

(6035, 20)

In [41]:
y_test.shape

(6035,)

In [42]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
12250/12250 [==============================] - 15s 1ms/sample - loss: 0.3435 - acc: 0.8380 - val_loss: 0.1956 - val_acc: 0.9142
Epoch 2/10
12250/12250 [==============================] - 12s 973us/sample - loss: 0.1527 - acc: 0.9376 - val_loss: 0.1941 - val_acc: 0.9205
Epoch 3/10
12250/12250 [==============================] - 11s 916us/sample - loss: 0.1095 - acc: 0.9562 - val_loss: 0.2168 - val_acc: 0.9094
Epoch 4/10
12250/12250 [==============================] - 12s 974us/sample - loss: 0.0894 - acc: 0.9678 - val_loss: 0.2631 - val_acc: 0.9138
Epoch 5/10
12250/12250 [==============================] - 11s 923us/sample - loss: 0.0722 - acc: 0.9745 - val_loss: 0.2688 - val_acc: 0.9130
Epoch 6/10
12250/12250 [==============================] - 11s 922us/sample - loss: 0.0594 - acc: 0.9783 - val_loss: 0.2944 - val_acc: 0.9080
Epoch 7/10
12250/12250 [==============================] - 1

**Performance Metrics and Accuracy**

In [43]:
y_pred=model.predict_classes(X_test)

In [44]:
y_pred[0]

array([1])

In [45]:
from sklearn.metrics import confusion_matrix

In [46]:
confusion_matrix(y_test,y_pred)

array([[3113,  306],
       [ 270, 2346]], dtype=int64)

In [47]:
from sklearn.metrics import accuracy_score

In [48]:
accuracy_score(y_test,y_pred)

0.9045567522783762